
# Задача

Вам даны 2 файла. В первом файле результат работы с сайтом пользователей, отобранных в контрольную группу, во втором - результат работы с сайтом пользователей, отобранных в экспериментальную группу, которой показывалась новая версия сайта.

У нас есть 4 поля, которые были выгружены из GoogleAnalytics:
* Pageviews - просмотры страниц
* Clicks - клики
* Enrollments - записи на курс
* Payments - оплаты курсов

Данные представлены в разрезе "по дням".

Ваша задача:
* Сформулировать гипотезы и метрики (что хотим померить).
* Протестировать гипотезы с помощью A/B test.
* Сделать выводы о том, какая из версий сайта лучше и почему.

In [22]:
import pandas as pd
import numpy as np
from scipy import stats

In [26]:
with open('control_data.csv') as f:
    df_control = pd.read_csv(f).fillna(0)
    
with open('experiment_data.csv') as f:
    df_exp = pd.read_csv(f).fillna(0)

In [27]:
df_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [28]:
df_exp.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [29]:
df_exp.shape == df_control.shape

True

### Гипотеза:
H0 - Среднее количество оплат одинакова для обеих групп
### Выбор порога значимости
Наблюдений не так много, поэтому примем, что $\alpha = 0.05$

In [30]:
np.mean(df_control['Payments'])

54.945945945945944

In [31]:
np.mean(df_exp['Payments'])

52.567567567567565

In [32]:
stats.normaltest(df_control['Payments'])

NormaltestResult(statistic=28.727416572283193, pvalue=5.779905683495005e-07)

In [33]:
stats.normaltest(df_exp['Payments'])

NormaltestResult(statistic=34.041698259069655, pvalue=4.054517187123e-08)

Используем тест стьюдента. Используем ЦПТ, т.к. изначальное распределение не нормально.

In [34]:
def get_c1_samp(paymets, size=10, iters=50):
    arr = []
    for i in range(iters):
        samp = np.random.choice(paymets, size=size)
        arr.append(np.mean(samp))
    return arr

In [35]:
exp_means = get_c1_samp(df_exp['Payments'])
control_means = get_c1_samp(df_control['Payments'])

In [36]:
stats.normaltest(exp_means)

NormaltestResult(statistic=1.9783186836714621, pvalue=0.3718891913355715)

In [37]:
stats.normaltest(control_means)

NormaltestResult(statistic=0.7472307083340339, pvalue=0.6882415901817525)

In [39]:
stats.ttest_ind(control_means, exp_means)

Ttest_indResult(statistic=1.130815052678299, pvalue=0.2608927723762357)

### Выводы

$T_{statistic} > 0$, значит новые изменения ухудшили результат.

$p_{value} >> \alpha$, значит мы не отвергаем Н0. Следовательно, новые изменения не лучше старого сайта с точки зрения выбранной метрики